# Regularyzacja w modelu regresji - zbiór `Hitter`

In [41]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 10

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# Zadanie - `Hitter`

Zbiór `Hitter` (pakiet `ISLR`) zawiera dane z **Major League Baseball** z sezonu 1986 i 1987, zawierają 322 obserwacje na temat głównych zawodników scharakteryzowanym na podstawie następujących zmiennych: 

``AtBat``
   Number of times at bat in 1986

``Hits``
   Number of hits in 1986

``HmRun``
   Number of home runs in 1986

``Runs``
   Number of runs in 1986

``RBI``
   Number of runs batted in in 1986

``Walks``
   Number of walks in 1986

``Years``
   Number of years in the major leagues

``CAtBat``
   Number of times at bat during his career

``CHits``
   Number of hits during his career

``CHmRun``
   Number of home runs during his career

``CRuns``
   Number of runs during his career

``CRBI``
   Number of runs batted in during his career

``CWalks``
   Number of walks during his career

``League``
   A factor with levels ``A`` and ``N`` indicating player's league at
   the end of 1986

``Division``
   A factor with levels ``E`` and ``W`` indicating player's division at
   the end of 1986

``PutOuts``
   Number of put outs in 1986

``Assists``
   Number of assists in 1986

``Errors``
   Number of errors in 1986

``Salary``
   1987 annual salary on opening day in thousands of dollars

``NewLeague``
   A factor with levels ``A`` and ``N`` indicating player's league at
   the beginning of 1987

1. Dopasuj model regresji liniowej, w którym zmienną zależną jest zmienna `Salary` a pozostałe cechy zmiennymi niezależnymi.
Wyznacz ocenę dopasowania modelu korzystając z kroswalidacji 10-krotnej dla następujących miar:
   - błędu średniokwadratowego, 
   - mediany błędu bezwzględnego, 
   - wspóczynnik determinacji $R^2$.
    
    
2. Dopasuj model regresji grzbietowej:

   a) dla dowolnie wybranego parametru $\alpha$ i wyznacz ocenę doapsowania modelu przy użyciu kroswalidacji 10-krotnej dla błędu średniokwadratowego, mediany błędu bezwzględnego, i wspóczynnik determinacji $R^2$.
   
   b) znajdź optymalną wartość parametru $\alpha$ (`GridSearchCV`)
   
   c) sporządź wykres wartości współczynników regresji względem parametru $\alpha$.
   

3. Dopasuj model regresji Lasso:

   a) dla dowolnie wybranego parametru $\alpha$ i wyznacz ocenę doapsowania modelu przy użyciu kroswalidacji 10-krotnej dla błędu średniokwadratowego, mediany błędu bezwzględnego, i wspóczynnik determinacji $R^2$.
   
   b) znajdź optymalną wartość parametru $\alpha$ (`GridSearchCV`)
   
   c) sporządź wykres wartości współczynników regresji względem parametru $\alpha$.

In [13]:
hitters = sm.datasets.get_rdataset(dataname="Hitters", package="ISLR", cache=True)
# print(hitters.__doc__)

/opt/anaconda3.7/lib/python3.7/site-packages/statsmodels/datasets/utils.py:192: FutureWarning: `item` has been deprecated and will be removed in a future version
  return dataset_meta["Title"].item()


In [14]:
hitters.data.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
-Andy Allanson,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [15]:
hitters.data.shape

(322, 20)

In [16]:
hitters.data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 322 entries, -Andy Allanson to -Willie Wilson
Data columns (total 20 columns):
AtBat        322 non-null int64
Hits         322 non-null int64
HmRun        322 non-null int64
Runs         322 non-null int64
RBI          322 non-null int64
Walks        322 non-null int64
Years        322 non-null int64
CAtBat       322 non-null int64
CHits        322 non-null int64
CHmRun       322 non-null int64
CRuns        322 non-null int64
CRBI         322 non-null int64
CWalks       322 non-null int64
League       322 non-null object
Division     322 non-null object
PutOuts      322 non-null int64
Assists      322 non-null int64
Errors       322 non-null int64
Salary       263 non-null float64
NewLeague    322 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 52.8+ KB


In [17]:
hitters = hitters.data.dropna()
hitters = hitters.drop(['League', 'Division', 'NewLeague'], axis = 1)

In [18]:
## Podział zbioru na zmienne niezależne i zmienną zależną
X, y = hitters.iloc[:, hitters.columns != 'Salary'], hitters['Salary']

In [ ]:
# wielokrotna regresja liniowa

In [19]:
lm = LinearRegression()
lm.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
results_linear_regression = {}
results_linear_regression["r2"] = lm.score(X, y)
results_linear_regression["mse"] = metrics.mean_squared_error(y_true=y, y_pred=lm.predict(X))
results_linear_regression["mae"] = metrics.median_absolute_error(y_true=y, y_pred=lm.predict(X))

In [22]:
results_linear_regression

{'r2': 0.5279308277431296, 'mse': 95704.59862968259, 'mae': 161.05945761299154}

In [23]:
# kroswalidacja
?cross_val_score

In [30]:
lm = LinearRegression()

results_linear_regression_cv = {}
results_linear_regression_cv["r2"] = cross_val_score(estimator=lm, X=X, y=y, scoring="r2", cv=10).mean()
results_linear_regression_cv["mse"] = -cross_val_score(estimator=lm, X=X, y=y, 
                                                      scoring="neg_mean_squared_error", cv=10).mean()
results_linear_regression_cv["mae"] = -cross_val_score(estimator=lm, X=X, y=y, 
                                                      scoring="neg_median_absolute_error", cv=10).mean()

In [31]:
results_linear_regression_cv

{'r2': 0.32613092426075657,
 'mse': 118756.28241460052,
 'mae': 173.2156746179618}

In [ ]:
# Ridge regression

In [32]:
pipeline_ridge = make_pipeline(
    StandardScaler(), 
    Ridge(alpha=0.1)
)

In [35]:
def calc_results(model):
    results_cv = {}
    results_cv["r2"] = cross_val_score(estimator=model, X=X, y=y, scoring="r2", cv=10).mean()
    results_cv["mse"] = -cross_val_score(estimator=model, X=X, y=y, 
                                                          scoring="neg_mean_squared_error", cv=10).mean()
    results_cv["mae"] = -cross_val_score(estimator=model, X=X, y=y, 
                                                      scoring="neg_median_absolute_error", cv=10).mean()
    return results_cv

In [37]:
results_ridge_cv = calc_results(pipeline_ridge)
results_ridge_cv

{'r2': 0.3301828107835725, 'mse': 118124.9490308509, 'mae': 169.83233977678435}

In [ ]:
# Lasso regression

In [43]:
?Lasso

In [38]:
pipeline_lasso = make_pipeline(
    StandardScaler(), 
    Lasso(alpha=0.1)
)

In [42]:
results_lasso_cv = calc_results(pipeline_lasso)
results_lasso_cv

{'r2': 0.32868914298309315,
 'mse': 118404.46100608057,
 'mae': 170.38083547991656}

## GridSearch

In [44]:
?GridSearchCV

In [46]:
alpha_vec = np.linspace(0.01, 10, 100)
alpha_vec

array([ 0.01      ,  0.11090909,  0.21181818,  0.31272727,  0.41363636,
        0.51454545,  0.61545455,  0.71636364,  0.81727273,  0.91818182,
        1.01909091,  1.12      ,  1.22090909,  1.32181818,  1.42272727,
        1.52363636,  1.62454545,  1.72545455,  1.82636364,  1.92727273,
        2.02818182,  2.12909091,  2.23      ,  2.33090909,  2.43181818,
        2.53272727,  2.63363636,  2.73454545,  2.83545455,  2.93636364,
        3.03727273,  3.13818182,  3.23909091,  3.34      ,  3.44090909,
        3.54181818,  3.64272727,  3.74363636,  3.84454545,  3.94545455,
        4.04636364,  4.14727273,  4.24818182,  4.34909091,  4.45      ,
        4.55090909,  4.65181818,  4.75272727,  4.85363636,  4.95454545,
        5.05545455,  5.15636364,  5.25727273,  5.35818182,  5.45909091,
        5.56      ,  5.66090909,  5.76181818,  5.86272727,  5.96363636,
        6.06454545,  6.16545455,  6.26636364,  6.36727273,  6.46818182,
        6.56909091,  6.67      ,  6.77090909,  6.87181818,  6.97

In [47]:
estimator = make_pipeline(
    StandardScaler(), 
    Ridge()
)
estimator

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ridge',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [49]:
grid_r2 = GridSearchCV(
    estimator=estimator, # model
    param_grid={'ridge__alpha': alpha_vec}, # ridge = krok z pipelinie, __ , parametr w wybranym kroku
    scoring="r2", # 
    cv=10
)

grid_r2.fit(X, y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('ridge',
                                        Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_state=None, solver='auto',
                                              tol=0.001))],
                                verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'ridge...
        7.07363636,  7.17454545,  7.27545455,  7.37636364,  7.47727273,
        7.57818182,  7.67909091,  7.78 

In [50]:
grid_r2.best_params_

{'ridge__alpha': 10.0}

In [ ]:
# zbudowac model na pelnym danych z wyznaczona alpha z grid search

In [51]:
ridge10 = make_pipeline(
    StandardScaler(), 
    Ridge(alpha = grid_r2.best_params_['ridge__alpha'])
)

In [52]:
ridge10.fit(X, y)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ridge',
                 Ridge(alpha=10.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [53]:
ridge10.score(X, y)

0.502616931945493